<a href="https://colab.research.google.com/github/icculp/route-66-scrape_public/blob/master/places.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import requests

In [2]:
columns = ['RoadTripStopTypeID',
'CategoryID',
'RoadTripStopName',
'RoadTripStopDescription',
'RoadTripStopAddressLine1',
'RoadTripStopCity',
'RoadTripStopState',
'RoadTripStopZip',
'RoadTripStopLat',
'RoadTripStopLong',
'RoadTripStopEmail',
'RoadTripStopPhone',
'TicketBookingURL',
'Image',
'EventDateTime',
'RoadTripStopURL',
'PlaceID',
'SunOpen',
'SunClose',
'MonOpen',
'MonClose',
'TueOpen',
'TueClose',
'WedOpen',
'WedClose', 
'ThurOpen',
'ThurClose',
'FriOpen',
'FriClose',
'SatOpen',
'SatClose']

You can either copy/paste the dictionary output from the script historic66 outputs, or load the data in from file. In colab you can upload to session storage

In [3]:
# places = {}
# print(len(places))

In [ ]:
!ls

In [ ]:
filename = 'historic66.json'
with open(filename, 'r', encoding='utf-8') as f:
    places = json.load(f)
print(len(places))
# dict(list(places.items())[0:5])  # preview places

In [13]:
key = ''  # leaving key out of repo for 

In [ ]:
Places_final = dict()
periods_default = dict()
i = 0
no_candidates = []
for Place, details in places.items():
    address = details['address']
    Description = details['description']
    Image = details['image']
    print(Place, end='')
    append = Place.replace(' ', '%20') + '%20' + address.replace(' ', '%20').replace('\n', '%20')
    pid = requests.get('https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=' + append + '&inputtype=textquery&key=' + key)
    try:
        Place_id = pid.json()['candidates'][0].get('place_id')
    except:
        no_candidates.append([Place, address, Description, Image])
        print("\nNo candidates for [{}], [{}], skipping".format(Place, address.replace('\n', ' ')))
        i += 1
        print(i, end='..')
        continue
    p =  requests.get('https://maps.googleapis.com/maps/api/place/details/json?place_id=' + Place_id + '&key=' + key)
    result = p.json()['result']

    adr_address = result.get('adr_address', 'NULL')
    soup = BeautifulSoup(adr_address, features="html.parser")
    Address = soup.find("span", {"class": "street-address"}).text if soup.find("span", {"class": "street-address"}) is not None else "NULL"
    City = soup.find("span", {"class": "locality"}).text if soup.find("span", {"class": "locality"}) is not None else "NULL"
    State = soup.find("span", {"class": "region"}).text if soup.find("span", {"class": "region"}) is not None else "NULL"
    Zip = soup.find("span", {"postal-code"}).text if soup.find("span", {"class": "postal-code"}) is not None else "NULL"

    Lat = str(result.get('geometry', 'NULL').get('location', 'NULL').get('lat', 'NULL'))
    Long = str(result.get('geometry', 'NULL').get('location', 'NULL').get('lng', 'NULL'))
    Email = 'NULL'
    Phone = result.get('formatted_phone_number', 'NULL')
    BookingUrl = result.get('website', 'NULL')
    URL = result.get('url', 'NULL')

    Periods = result.get('opening_hours')
    days = {j: {'open': 'NULL', 'close': 'NULL'} for j in range(7)}
    if Periods is not None:
        for day in Periods['periods']: 
            for k, v in day.items():
                timeForm = v.get('time')
                timeForm = timeForm[:2] + ':' + timeForm[2:]
                days[v['day']].update({k: timeForm})
    hoo = []  # hours of operation
    for k, d in days.items():
        hoo.append(d.get('open'))
        hoo.append(d.get('close'))

    line_items = [str(2), 'NULL', Place, Description, Address, City, State, Zip, Lat, Long, Email, Phone, BookingUrl, Image, 'NULL', URL, Place_id]
    line_items += hoo  # [SunOpen, SunClose, MonOpen, MonClose, TueOpen, TueClose, WedOpen, WedClose, ThurOpen, ThurClose, FriOpen, FriClose, SatOpen, SatClose]
    Places_final.update({Place_id: line_items})

    i += 1
    #if i == 5:
    #    break
    print(i, end='..')

In [ ]:
print("No candidates found for the following items, totalling {} places".format(len(no_candidates)))
no_candidates

In [ ]:
# dict(list(Places_final.items())[0:5])  # preview places

In [ ]:
Lines_final = list()

for plc_id, plc_line in Places_final.items():
    '''line_string = ''
    for item in range(len(plc_line)):
        line_string += str(plc_line[item])
        if item != len(plc_line) - 1:
            line_string += '|'''
    Lines_final.append(plc_line)
# Lines_final

In [ ]:
df_final = pd.DataFrame(Lines_final, columns=columns)
print(len(df_final))
df_final

In [ ]:
fileName = 'places_final.txt'
df_final.to_csv(fileName, sep="|", quoting=csv.QUOTE_NONNUMERIC, index=False)

In [ ]:
!ls